<a href="https://colab.research.google.com/github/nikadilli/elemental-imaging-enhancement/blob/master/UNetSR_test_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/nikadilli/elemental-imaging-enhancement

Cloning into 'elemental-imaging-enhancement'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 147 (delta 63), reused 102 (delta 32), pack-reused 0
Receiving objects: 100% (147/147), 9.45 MiB | 29.96 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [6]:
cd elemental-imaging-enhancement

/content/elemental-imaging-enhancement


In [0]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mem import *

from torchvision.models import vgg16_bn
from misc import prediction_to_image



In [0]:
width = 300 #@param {type:"number"}
height = 300 #@param {type:"number"}
path = "./test_data" #@param {type:"string"}
LR_folder = "cellsLR" #@param {type:"string"}
HR_folder = "cellsHR" #@param {type:"string"}
test_folder = "test" #@param {type:"string"}

In [0]:
size = (width, height)
path = Path(path)

In [58]:
!mkdir models
!wget https://www.dropbox.com/s/aa7fqv4sypb0ecy/model.pth -O ./models/m1.pth

mkdir: cannot create directory ‘models’: File exists
--2019-11-20 10:25:49--  https://www.dropbox.com/s/aa7fqv4sypb0ecy/model.pth
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/aa7fqv4sypb0ecy/model.pth [following]
--2019-11-20 10:25:49--  https://www.dropbox.com/s/raw/aa7fqv4sypb0ecy/model.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9256add82669d31ddf447dd004.dl.dropboxusercontent.com/cd/0/inline/AsvJAXMvX_eDgUr0srXZL_2X7vyKqtvviIGuO54k-ec1Prvw3yG7foDopJVZTVkiQ1gMKw-QYytjDpUALf---dc5-kY-zWq8lUNd3i_3piMhvwYxiPuX_cmjCzMTd2EXmiU/file# [following]
--2019-11-20 10:25:49--  https://uc9256add82669d31ddf447dd004.dl.dropboxusercontent.com/cd/0/inline/AsvJAXMvX_eDgUr0srXZL_2X7vyKqtvviIGuO54k-ec1Prvw3yG7foDopJVZTVkiQ1gMKw

In [71]:
data = (ImageImageList.from_folder(path/LR_folder).split_by_rand_pct(.5, seed=42)
          .label_from_func(lambda x: path/HR_folder/x.name)
          #.add_test_folder(test_folder)
          .transform(get_transforms(), size=size, tfm_y=True)
          .databunch(bs=1).normalize(imagenet_stats, do_y=True))
data.c = 3

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


OSError: ignored